# CMPT 459 Assignment 1

### *Name -> Harman Singh*
### *ID -> 301326898*
---

## Imports

In [1]:
# Standard Library
import string
import re
import sys
from collections import Counter
import unicodedata

# Third Party
import pandas as pdsorted(dic, key=operator.itemgetter(1))
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from langdetect import detect
from texttable import Texttable
from matplotlib_venn_wordcloud import venn2_wordcloud

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')

SyntaxError: invalid syntax (<ipython-input-1-9a1aa3820396>, line 9)

## Helper Functions

In [ ]:
def filter_text(text):
    """This helper method removes links, RT, hashtags, punctuations from text
    """
    # Remove RT @user: user
    text = re.sub(r'(^RT )?@[A-Za-z0-9]+(:)?', "", text)
    
    # Remove html tags like &amp
    text = re.sub(r'&[a-z]+', '', text)
    
    # Remove links
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+',"", text)
    
    # Lowercase Everything
    text = text.lower()
    
    # https://stackoverflow.com/a/33417311
    # Remove emojis
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # Remove unicode punctuation
    text = remove_punctuation(text)
    
    # Get rid of all spaces in the beginning or end
    text = text.strip()
    return text

def number_of_unique_tokens(tokens):
    counter = Counter(tokens)
    return len(counter)


def print_most_frequent(_dict, n):
    table = Texttable()
    table.header(['Token', 'Normalized Frequency'])
    for word, freq in _dict.most_common(n):
        table.add_row([word, freq/1000])
    print(table.draw())


# Taken from TA's video
def detect_language(token):
    try:
        return detect(token)
    except:
        return 'Other'
    

stop_words = set(stopwords.words('english'))
def remove_stop_words(text):
    """ This functions filters out all english stop words from text
    """
    tokenized = word_tokenize(text)
    filtered = [
        token for token in tokenized if (
            len(token) > 2 and
            token not in stop_words and
            token.isalnum() and
            token != 'nan'
        )
    ]
    return filtered


# https://stackoverflow.com/a/11066687
# Helper for removing unicode punctuation
tbl = dict.fromkeys(
    i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P')
)
def remove_punctuation(text):
    return text.translate(tbl)

## Load CSV Data

In [ ]:
df1 = pd.read_csv('D1.csv')
df2 = pd.read_csv('D2.csv')

## Preprocess Data to remove all punctuations

In [ ]:
# Process D1
df1['filtered'] = df1['text'].apply(filter_text)
df1['lang'] = df1['filtered'].apply(detect_language)
df1 = df1[df1['lang'] == 'en']
df1['without_stop_words'] = df1['filtered'].apply(remove_stop_words)

# Process D2
df2['filtered'] = df2['text'].apply(filter_text)
df2['lang'] = df2['filtered'].apply(detect_language)
df2 = df2[df2['lang'] == 'en']
df2['without_stop_words'] = df2['filtered'].apply(remove_stop_words)


## Q 3.1 Number of unique tokens

In [2]:
unique_tokens_D1 = number_of_unique_tokens(df1['without_stop_words'].explode().unique())
unique_tokens_D2 = number_of_unique_tokens(df2['without_stop_words'].explode().unique())

print(f'Number of Unique Tokens in D1: {unique_tokens_D1}, Number of Unique Tokens in D2: {unique_tokens_D2}')

NameError: name 'number_of_unique_tokens' is not defined

## Q 3.2 Top-100 most frequent tokenss

In [ ]:
freq_D1 = nltk.FreqDist(list(df1['without_stop_words'].explode()))
print('100 Most Frequent Tokens in D1:')
print_most_frequent(freq_D1, 100)

freq_D2 = nltk.FreqDist(list(df2['without_stop_words'].explode()))
print('100 Most Frequent Tokens in D2:')
print_most_frequent(freq_D2, 100)

## Q 3.3 Word Cloud

In [ ]:
# For this part I use python word cloud library to create word cloud from frequency dictionary generated from nltk
# Word Cloud that gets generated has random colors but the relative size of word is bigger for more frequent word

wc = WordCloud()
fig = plt.figure(figsize=(30, 30))

# Plot D1
wc.generate_from_frequencies(freq_D1)
fig.add_subplot(1, 2, 1)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Words from random tweets (D1)", fontSize=24)

# Plot D2
wc.generate_from_frequencies(freq_D2)
fig.add_subplot(1, 2, 2)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Words from covid tweets (D2)", fontSize=24)

plt.show(block=True)
fig.tight_layout()

## Q 3.4 Comparing datasets

In [ ]:
# For comparing the words from D1 and D2, I chose to draw Venn Diagram.
# Sizes are random, since I am not passing frequency
# (there were some bugs with library when I passed frequency)

fig, axes = plt.subplots(figsize=(20, 20))
axes.set_title('Comparing Top 50 Words from D1 and D2', fontsize=20)

# This libray creates venn diagram and fill them with word cloud
venn = venn2_wordcloud(
    [
        set(key for key, _ in freq_D1.most_common(50)),
        set(key for key, _ in freq_D2.most_common(50))
    ],
    ax=axes,
    set_labels=['D1', 'D2'],
)

# Set Colors
venn.get_patch_by_id('10').set_color('blue')
venn.get_patch_by_id('10').set_alpha(0.4)
venn.get_patch_by_id('01').set_color('red')
venn.get_patch_by_id('01').set_alpha(0.4)
venn.get_patch_by_id('11').set_color('purple')
venn.get_patch_by_id('11').set_alpha(0.4)


## Q 3.5 Link to Twitter

## [Link to Tweet](https://twitter.com/iamgrewal7/status/1267226701236334592?s=20 "Twitter")
---